In [1]:
import os
import random

In [2]:
def mkdir(path):
    '''
    创建指定的文件夹
    :param path: 文件夹路径，字符串格式
    :return: True(新建成功) or False(文件夹已存在，新建失败)
    '''
    # 引入模块
    import os

    # 去除首位空格
    path = path.strip()
    # 去除尾部 \ 符号
    path = path.rstrip("\\")

    # 判断路径是否存在
    # 存在     True
    # 不存在   False
    isExists = os.path.exists(path)

    # 判断结果
    if not isExists:
        # 如果不存在则创建目录
         # 创建目录操作函数
        os.makedirs(path)
        print(path + ' 创建成功')
        return True
    else:
        # 如果目录存在则不创建，并提示目录已存在
        print(path + ' 目录已存在')
        return False

In [7]:
# lepton = 'electron'#'muon' or 'electron'
# isdata = 'data'#'data' or 'mc'
# target_dir = f'parquet_{isdata}_{lepton}_fake_rate'
wrong_num = 0
for lepton in ['electron','muon']:
    for isdata in ['data','mc']:
# for lepton in ['electron']:
#     for isdata in ['data']:
        target_dir = f'coffea_parquet_{isdata}_{lepton}_fake_rate'
        if isdata=='mc':
            MC_list = ['WJets','DYJetsToLL_M-10to50','DYJetsToLL_M-50']
        else:
            MC_list = [1]
        for MC in MC_list:
            if lepton=='electron' and isdata=='data':
                data_nanoAOD_dir_list = ['/data/pubfs/pku_visitor/public_write/download/datasets/nanov7/EGamma/Run2018A-02Apr2020-v1/NANOAOD',
                                        '/data/pubfs/pku_visitor/public_write/download/datasets/nanov7/EGamma/Run2018B-02Apr2020-v1/NANOAOD',
                                        '/data/pubfs/pku_visitor/public_write/download/datasets/nanov7/EGamma/Run2018C-02Apr2020-v1/NANOAOD',
                                        '/data/pubfs/pku_visitor/public_write/download/datasets/nanov7/EGamma/Run2018D-02Apr2020-v1/NANOAOD']
            elif lepton=='muon' and isdata=='data':
                data_nanoAOD_dir_list = ['/data/pubfs/pku_visitor/public_write/download/datasets/nanov7/DoubleMuon/Run2018A-02Apr2020-v1/NANOAOD',
                                        '/data/pubfs/pku_visitor/public_write/download/datasets/nanov7/DoubleMuon/Run2018B-02Apr2020-v1/NANOAOD',
                                        '/data/pubfs/pku_visitor/public_write/download/datasets/nanov7/DoubleMuon/Run2018C-02Apr2020-v1/NANOAOD',
                                        '/data/pubfs/pku_visitor/public_write/download/datasets/nanov7/DoubleMuon/Run2018D-02Apr2020-v1/NANOAOD']
            else:
                if isdata=='mc':
                    if MC=='WJets':
                        data_nanoAOD_dir_list = ['/data/pubfs/pku_visitor/public_write/download/datasets/nanov7/WJetsToLNu_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM']
                    elif MC=='DYJetsToLL_M-50':
                        data_nanoAOD_dir_list = ['/data/pubfs/pku_visitor/public_write/download/datasets/nanov7/DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM']
                    elif MC=='DYJetsToLL_M-10to50': 
                        data_nanoAOD_dir_list = ['/data/pubfs/pku_visitor/public_write/download/datasets/nanov7/DYJetsToLL_M-10to50_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM']
                    else:
                        print('wrong MC name')
                        data_nanoAOD_dir = ''
                else:
                    print('!!!wrong lepton type!!!')
            if isdata=='data':
                output_dir = target_dir
            else:
                output_dir = target_dir + f'/{MC}'
            py_file = 'fake_rate_parquet_coffea.py'

            pwd = os.popen('pwd').readlines()[0].replace('\n','')

            mkdir(output_dir)

            total_num_file = 0
            for data_nanoAOD_dir in data_nanoAOD_dir_list:
                for i,rootfile in enumerate(os.listdir(data_nanoAOD_dir)):
                    total_num_file += 1

            nfile = 0
            for data_nanoAOD_dir in data_nanoAOD_dir_list:
                for i,rootfile in enumerate(os.listdir(data_nanoAOD_dir)):
                    nfile += 1
                    if nfile<999:
                        if not os.path.exists(pwd+'/'+output_dir+'/'+str(nfile)+'/'+'events.parquet'):
                            wrong_num += 1
                            print('wrong %d: %s\t%s\t%d'%(wrong_num,isdata,lepton,nfile))
                            mkdir("%s/%d"%(output_dir,nfile))
                            nanoAOD_rootfiles = data_nanoAOD_dir+'/'+rootfile
                            print("\n%d/%d"%(nfile,total_num_file))
                            command = 'python %s -n $2 -y 2018 -f $1 -t %s -o %s/%d -l %s'%(pwd+'/'+py_file, isdata, pwd+'/'+output_dir, nfile, lepton)
                            file_command = open('%s/%d/command_%d.sh'%(output_dir,nfile,nfile),'w')
                            file_command.write('#! /bin/bash\n\n')
                            file_command.write('sleep %d\n'%(random.randint(10,900)))
                            file_command.write('source /home/pku/tyyang99/.bashrc\n')
                            file_command.write('cd %s\n'%(pwd))
                            file_command.write(command+'\n')
                            file_command.write('echo "job done!"')
                            file_command.close()

                            file_submit = open('%s/%d/submit_%d.txt'%(output_dir,nfile,nfile),'w')
                            file_submit.write('Universe = vanilla\n')
                            file_submit.write('Executable = %s/%d/command_%d.sh\n'%(output_dir,nfile,nfile))
                            file_submit.write('requirements = (machine == "node05.phy.pku.edu.cn") || (machine == "node06.phy.pku.edu.cn")\n')
                            file_submit.write('arguments = %s %d\n'%(nanoAOD_rootfiles,nfile))
                            file_submit.write('should_transfer_files = YES\n')
                            file_submit.write('Error = %s/%d/%d.err\n'%(output_dir,nfile,nfile))
                            file_submit.write('Output = %s/%d/%d.out\n'%(output_dir,nfile,nfile))
                            file_submit.write('Log = %s/%d/%d.log\n'%(output_dir,nfile,nfile))
                            file_submit.write('Queue 1 \n')
                            file_submit.close()

                            condor_output = os.popen('condor_submit %s/%d/submit_%d.txt'%(output_dir,nfile,nfile)).readlines()
                            for line in condor_output:
                                print(line,end='')

coffea_parquet_data_electron_fake_rate 目录已存在
wrong 1: data	electron	295
coffea_parquet_data_electron_fake_rate/295 目录已存在

295/946
Submitting job(s).
1 job(s) submitted to cluster 135826.
wrong 2: data	electron	334
coffea_parquet_data_electron_fake_rate/334 目录已存在

334/946
Submitting job(s).
1 job(s) submitted to cluster 135827.
wrong 3: data	electron	340
coffea_parquet_data_electron_fake_rate/340 目录已存在

340/946
Submitting job(s).
1 job(s) submitted to cluster 135828.
wrong 4: data	electron	384
coffea_parquet_data_electron_fake_rate/384 目录已存在

384/946
Submitting job(s).
1 job(s) submitted to cluster 135829.
wrong 5: data	electron	402
coffea_parquet_data_electron_fake_rate/402 目录已存在

402/946
Submitting job(s).
1 job(s) submitted to cluster 135830.
wrong 6: data	electron	404
coffea_parquet_data_electron_fake_rate/404 目录已存在

404/946
Submitting job(s).
1 job(s) submitted to cluster 135831.
wrong 7: data	electron	410
coffea_parquet_data_electron_fake_rate/410 目录已存在

410/946
Submitting job(s).

Submitting job(s).
1 job(s) submitted to cluster 135882.
wrong 58: data	muon	177
coffea_parquet_data_muon_fake_rate/177 目录已存在

177/238
Submitting job(s).
1 job(s) submitted to cluster 135883.
wrong 59: data	muon	184
coffea_parquet_data_muon_fake_rate/184 目录已存在

184/238
Submitting job(s).
1 job(s) submitted to cluster 135884.
wrong 60: data	muon	191
coffea_parquet_data_muon_fake_rate/191 目录已存在

191/238
Submitting job(s).
1 job(s) submitted to cluster 135885.
wrong 61: data	muon	192
coffea_parquet_data_muon_fake_rate/192 目录已存在

192/238
Submitting job(s).
1 job(s) submitted to cluster 135886.
wrong 62: data	muon	199
coffea_parquet_data_muon_fake_rate/199 目录已存在

199/238
Submitting job(s).
1 job(s) submitted to cluster 135887.
wrong 63: data	muon	201
coffea_parquet_data_muon_fake_rate/201 目录已存在

201/238
Submitting job(s).
1 job(s) submitted to cluster 135888.
wrong 64: data	muon	209
coffea_parquet_data_muon_fake_rate/209 目录已存在

209/238
Submitting job(s).
1 job(s) submitted to cluster 135889

In [18]:
[[[False] * 9 for _a in range(3)] for _b in range(3)]

[[[False, False, False, False, False, False, False, False, False],
  [False, False, False, False, False, False, False, False, False],
  [False, False, False, False, False, False, False, False, False]],
 [[False, False, False, False, False, False, False, False, False],
  [False, False, False, False, False, False, False, False, False],
  [False, False, False, False, False, False, False, False, False]],
 [[False, False, False, False, False, False, False, False, False],
  [False, False, False, False, False, False, False, False, False],
  [False, False, False, False, False, False, False, False, False]]]

In [1]:
import awkward as ak

test_array = ak.from_parquet('coffea_parquet_data_electron_fake_rate/1/events.parquet')

In [14]:
test_array.fields

['Lepton_deltaEtaSC',
 'Lepton_dr03EcalRecHitSumEt',
 'Lepton_dr03HcalDepth1TowerSumEt',
 'Lepton_dr03TkSumPt',
 'Lepton_dr03TkSumPtHEEP',
 'Lepton_dxy',
 'Lepton_dxyErr',
 'Lepton_dz',
 'Lepton_dzErr',
 'Lepton_eCorr',
 'Lepton_eInvMinusPInv',
 'Lepton_energyErr',
 'Lepton_eta',
 'Lepton_hoe',
 'Lepton_ip3d',
 'Lepton_jetPtRelv2',
 'Lepton_jetRelIso',
 'Lepton_mass',
 'Lepton_miniPFRelIso_all',
 'Lepton_miniPFRelIso_chg',
 'Lepton_mvaFall17V1Iso',
 'Lepton_mvaFall17V1noIso',
 'Lepton_mvaFall17V2Iso',
 'Lepton_mvaFall17V2noIso',
 'Lepton_pfRelIso03_all',
 'Lepton_pfRelIso03_chg',
 'Lepton_phi',
 'Lepton_pt',
 'Lepton_r9',
 'Lepton_scEtOverPt',
 'Lepton_sieie',
 'Lepton_sip3d',
 'Lepton_mvaTTH',
 'Lepton_charge',
 'Lepton_cutBased',
 'Lepton_cutBased_Fall17_V1',
 'Lepton_jetIdx',
 'Lepton_pdgId',
 'Lepton_photonIdx',
 'Lepton_tightCharge',
 'Lepton_vidNestedWPBitmap',
 'Lepton_vidNestedWPBitmapHEEP',
 'Lepton_convVeto',
 'Lepton_cutBased_HEEP',
 'Lepton_isPFcand',
 'Lepton_lostHits',
 '

In [8]:
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema, BaseSchema, TreeMakerSchema

events = NanoEventsFactory.from_parquet('coffea_parquet_data_electron_fake_rate/1/events.parquet',schemaclass=NanoAODSchema).events()

/home/pku/tyyang99/anaconda3/lib/python3.8/site-packages/coffea/nanoevents/schemas/nanoaod.py:193: RuntimeWarning: Missing cross-reference index for Electron_jetIdx => Jet
  warnings.warn(
/home/pku/tyyang99/anaconda3/lib/python3.8/site-packages/coffea/nanoevents/schemas/nanoaod.py:193: RuntimeWarning: Missing cross-reference index for Electron_photonIdx => Photon
  warnings.warn(
/home/pku/tyyang99/anaconda3/lib/python3.8/site-packages/coffea/nanoevents/schemas/nanoaod.py:193: RuntimeWarning: Missing cross-reference index for FatJet_subJetIdx1 => SubJet
  warnings.warn(
/home/pku/tyyang99/anaconda3/lib/python3.8/site-packages/coffea/nanoevents/schemas/nanoaod.py:193: RuntimeWarning: Missing cross-reference index for FatJet_subJetIdx2 => SubJet
  warnings.warn(
/home/pku/tyyang99/anaconda3/lib/python3.8/site-packages/coffea/nanoevents/schemas/nanoaod.py:193: RuntimeWarning: Missing cross-reference index for FsrPhoton_muonIdx => Muon
  warnings.warn(
/home/pku/tyyang99/anaconda3/lib/pyt

In [9]:
events.fields

['Lepton', 'PuppiMET', 'MET', 'HLT', 'Jet', 'lepton']

In [13]:
events['MET_pt']

ValueError: key "MET_pt" does not exist (not in record)

(https://github.com/scikit-hep/awkward-1.0/blob/1.5.1/src/libawkward/util.cpp#L525)

In [14]:
events.HLT.fields

['Mu12_TrkIsoVVL_Ele23_CaloIdL_TrackIdL_IsoVL_DZ',
 'Mu12_TrkIsoVVL_Ele23_CaloIdL_TrackIdL_IsoVL',
 'Mu23_TrkIsoVVL_Ele12_CaloIdL_TrackIdL_IsoVL_DZ',
 'Mu23_TrkIsoVVL_Ele12_CaloIdL_TrackIdL_IsoVL',
 'Mu8_TrkIsoVVL_Ele23_CaloIdL_TrackIdL_IsoVL_DZ',
 'Mu8_TrkIsoVVL_Ele23_CaloIdL_TrackIdL_IsoVL',
 'Mu17_TrkIsoVVL_Mu8_TrkIsoVVL_DZ_Mass3p8',
 'Mu17_TrkIsoVVL_Mu8_TrkIsoVVL_DZ_Mass8',
 'Mu19_TrkIsoVVL_Mu9_TrkIsoVVL_DZ_Mass3p8',
 'Mu19_TrkIsoVVL_Mu9_TrkIsoVVL_DZ_Mass8',
 'Ele23_Ele12_CaloIdL_TrackIdL_IsoVL_DZ',
 'Ele23_Ele12_CaloIdL_TrackIdL_IsoVL',
 'DiEle27_WPTightCaloOnly_L1DoubleEG',
 'DoubleEle33_CaloIdL_MW',
 'DoubleEle25_CaloIdL_MW',
 'DoubleEle27_CaloIdL_MW',
 'DoublePhoton70',
 'IsoMu24',
 'IsoMu27',
 'IsoMu30',
 'Mu50',
 'Ele115_CaloIdVT_GsfTrkIdT',
 'Ele27_WPTight_Gsf',
 'Ele32_WPTight_Gsf',
 'Ele35_WPTight_Gsf',
 'Ele38_WPTight_Gsf',
 'Ele40_WPTight_Gsf',
 'Ele32_WPTight_Gsf_L1DoubleEG',
 'Photon200',
 'Mu8_TrkIsoVVL',
 'Mu17_TrkIsoVVL',
 'Ele8_CaloIdL_TrackIdL_IsoVL_PFJet30',
 'El

In [6]:
os.path.exists('/data/pubfs/tyyang99/jupyter_files/pkutree/coffea_parquet_data_electron_fake_rate/762/events.parquet')

True

In [5]:
pwd+'/'+output_dir+'/'+str(761+1)+'/'+'events.parquet'

'/data/pubfs/tyyang99/jupyter_files/pkutree/coffea_parquet_mc_muon_fake_rate/DYJetsToLL_M-50/762/events.parquet'